In [1]:
import json
import re
from pathlib import Path

import pytest

from datasets.QuixBugs.python_testcases.load_testdata import load_json_testcases
from utils import get_all_function, copy_to_custom_folder

root_path = Path('/Users/lucyxu/PycharmProjects/debug/')
target_path = root_path / 'datasets' / 'QuixBugs' / 'python_testcases'

load_json_testcases('bitcount')


[[[127], 7],
 [[128], 1],
 [[3005], 9],
 [[13], 3],
 [[14], 3],
 [[27], 4],
 [[834], 4],
 [[254], 7],
 [[256], 1]]

In [2]:
function_list = get_all_function()

generated_code_path = Path('generated_code') / 'gpt35'
copy_to_custom_folder(generated_code_path, 1)



In [3]:
import sys
import io

stdout_bak = sys.stdout  # backup stdout
sys.stdout = io.StringIO()

# pytest.main(['--custom', 'python_testcases'])
pytest.main(['--custom', target_path])

output = sys.stdout.getvalue()
sys.stdout.close()
sys.stdout = stdout_bak  # restore stdout

print(output)


============================= test session starts ==============================
platform darwin -- Python 3.9.6, pytest-7.4.3, pluggy-1.3.0
rootdir: /Users/lucyxu/PycharmProjects/debug
plugins: anyio-3.7.1
collected 278 items

datasets/QuixBugs/python_testcases/test_bitcount.py .........            [  3%]
datasets/QuixBugs/python_testcases/test_breadth_first_search.py .....    [  5%]
datasets/QuixBugs/python_testcases/test_bucketsort.py .......            [  7%]
datasets/QuixBugs/python_testcases/test_depth_first_search.py .....      [  9%]
datasets/QuixBugs/python_testcases/test_detect_cycle.py FFFFFF           [ 11%]
datasets/QuixBugs/python_testcases/test_find_first_in_sorted.py .......  [ 14%]
datasets/QuixBugs/python_testcases/test_find_in_sorted.py .......        [ 16%]
datasets/QuixBugs/python_testcases/test_flatten.py .......               [ 19%]
datasets/QuixBugs/python_testcases/test_gcd.py ......                    [ 21%]
datasets/QuixBugs/python_testcases/test_get_factors.

In [4]:
# 7-bit C1 ANSI sequences
ansi_escape = re.compile(r'''
    \x1B  # ESC
    (?:   # 7-bit C1 Fe (except CSI)
        [@-Z\\-_]
    |     # or [ for CSI, followed by a control sequence
        \[
        [0-?]*  # Parameter bytes
        [ -/]*  # Intermediate bytes
        [@-~]   # Final byte
    )
''', re.VERBOSE)
progress_text = re.compile(r'\[[\s\d]+%\]')
text_divide = re.compile(r'''
    (datasets/QuixBugs/python_testcases/test_(\w+).py) 
    \s+
    ([.Fs]+)
''', re.VERBOSE)

output_copy = output
output_copy = ansi_escape.sub('', output_copy)
output_copy = progress_text.sub('', output_copy)

start_text = re.compile(r'.*=+ test session starts =+.*')
start_index = start_text.search(output_copy).start()

end_text = re.compile(r'=+.* FAILURES =+.*')
end_index = end_text.search(output_copy).end()

splitted_output = output_copy[start_index:end_index]
splitted_output = splitted_output.split('\n')
splitted_output = splitted_output[6:-2]

temp = []
for i in range(len(splitted_output)):
    line = splitted_output[i]
    if line.startswith('datasets/QuixBugs/python_testcases/'):
        temp.append(line)
    else:
        temp[-1] = temp[-1].strip() + line.strip()
splitted_output = temp

parsed_output = splitted_output
parsed_output = [text_divide.match(line).groups() for line in parsed_output]

# Check divided output
pass_count = 0
fail_count = 0
skip_count = 0
result = []
for _, function_name, text in parsed_output:
    pass_count += text.count('.')
    fail_count += text.count('F')
    skip_count += text.count('s')
    
    result.append({
        'function_name': function_name,
        'test_text': text,
        'test_result': 'pass' if text.count('F') == 0 else 'fail',
        'fail_test_case': [m.start() for m in re.finditer('F', text)]
    })

observe_count = (fail_count, pass_count, skip_count)
real_count_pattern = re.compile(r'''
    =+
    \s(\d+)\s failed,
    \s(\d+)\s passed,
    \s(\d+)\s skipped
''', re.VERBOSE)
real_count = tuple(map(int, real_count_pattern.match(output_copy.split('\n')[-2]).groups()))

# output_copy.split('\n')[-2]
    
# print('divided: ', f'================== {fail_count} failed, {pass_count} passed, {skip_count} skipped ===================')
# print('origin:  ', output.split('\n')[-2])

# print(json.dumps(result, indent=4))


In [6]:
from utils import extract_test_info_from_pytest_log

result2 = extract_test_info_from_pytest_log(output)


[{'function_name': 'bitcount',
  'test_text': '.........',
  'test_result': 'pass',
  'fail_test_case': []},
 {'function_name': 'breadth_first_search',
  'test_text': '.....',
  'test_result': 'pass',
  'fail_test_case': []},
 {'function_name': 'bucketsort',
  'test_text': '.......',
  'test_result': 'pass',
  'fail_test_case': []},
 {'function_name': 'depth_first_search',
  'test_text': '.....',
  'test_result': 'pass',
  'fail_test_case': []},
 {'function_name': 'detect_cycle',
  'test_text': 'FFFFFF',
  'test_result': 'fail',
  'fail_test_case': [0, 1, 2, 3, 4, 5]},
 {'function_name': 'find_first_in_sorted',
  'test_text': '.......',
  'test_result': 'pass',
  'fail_test_case': []},
 {'function_name': 'find_in_sorted',
  'test_text': '.......',
  'test_result': 'pass',
  'fail_test_case': []},
 {'function_name': 'flatten',
  'test_text': '.......',
  'test_result': 'pass',
  'fail_test_case': []},
 {'function_name': 'gcd',
  'test_text': '......',
  'test_result': 'pass',
  'fail_te